# Playing with MNIST

This notebook expects you to have previously trained the MNIST model and saved the resulting file. 

## Canvas Installation: Two Workflows

### 1. Jupyter (locally)

The recommended way is to clone the repo, which contains `canvas.py`. Make sure you have [pycairo](https://anaconda.org/conda-forge/pycairo) installed:

```bash
conda activate dmlap
conda install -c conda-forge pycairo
```

### 2. Google Colab

When using Google Colab you will need to use `pip` and install additional libraries (based on [this](https://github.com/pygobject/pycairo/issues/39#issuecomment-391830334)):

```bash
# WARNING!!!! Do NOT do this if you are running jupyter/python locally!!!
!apt-get install libcairo2-dev libjpeg-dev libgif-dev
!pip install pycairo
```

#### 2.1 Working with the repo in your drive

Mount your drive and change to the correct directory:

```python
from google.colab import drive
drive.mount('/content/drive')

# change directory using the os module
import os
os.chdir('drive/My Drive/')
os.listdir()             # shows the contents of the current dir, you can use chdir again after that
# os.mkdir("DMLCP-2023") # creating a directory
# os.chdir("DMLCP-2023") # moving to this directory
# os.getcwd()            # printing the current directory
```

See [this notebook](https://colab.research.google.com/notebooks/io.ipynb), and [Working With Files](https://realpython.com/working-with-files-in-python/) on Real Python.

#### 2.2 Working on it as a standalone notebook

Get the`canvas` module:

```python
!curl -O https://raw.githubusercontent.com/jchwenger/DMLCP/main/python/canvas.py
```

Download and unzip the necessary images with:

```python
!curl -O https://raw.githubusercontent.com/jchwenger/DMLCP/main/python/images/3.png
!curl -O https://raw.githubusercontent.com/jchwenger/DMLCP/main/python/images/4.png
!mkdir images
!mv 3.png 4.png images
```

In [ ]:
import canvas
import pathlib
from PIL import Image

import numpy as np

import torch
from torch import nn
import torch.nn.functional as F

import torchvision as tv
from torchvision.transforms import v2

# Get cpu, gpu or mps device for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

## Load a trained network

In [ ]:
NUM_CLASSES = 10
INPUT_SHAPE = [1,28,28]

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten() # [1, 28, 28] -> [1, 28*28]
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(INPUT_SHAPE[1] * INPUT_SHAPE[2], 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, NUM_CLASSES)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

MODELS_DIR = pathlib.Path("models")

MODEL_NAME = "dense_mnist"

MNIST_DIR = MODELS_DIR / MODEL_NAME

model = NeuralNetwork().to(device)
model.load_state_dict(torch.load(MNIST_DIR / f"{MODEL_NAME}.pt", weights_only=True))

## Classify an image of a number

In [ ]:
img = Image.open('images/3.png') # try also images/4.png

transforms = tv.transforms.v2.Compose([  
    tv.transforms.Grayscale(num_output_channels=1),
    tv.transforms.Resize(size=(28,28), antialias=True),
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True)
])

input = transforms(img)
input = input.to(device)

print(f"Input shape: {input.shape}")

def predict(model, input): 
    model.eval()
    with torch.no_grad():
        probs = nn.Softmax(dim=-1)(model(input)).cpu().numpy()
        return np.argmax(probs[0])
        
predicted = predict(model, input)
canvas.show_image(img, title=f'Predicted number: {predicted}', cmap='gray')

## Two directions

1. **disrupt**: try and find cases where the network fails to predict the images properly
2. **generate**: come up with your own images and try to classify them! Combining the two, you can try to generate images that the network fails to classify!

### Note: Dense vs ConvNet

If you tried to train a ConvNet, you will notice that it tends to be more stable in its prediction!

## 1. Disrupt

Here we provide you with a canvas object that generates images with a number. You can see that a Dense net not always succeeds (and the ConvNet does).

In [ ]:
# Generate a random number between 0 and 9 (the max is excluded)
number = np.random.randint(0, 10) 
c = canvas.Canvas(28, 28)
c.background(0)
c.fill(255)
c.text_size(26)
c.text([c.width/2, c.height/2 + 9], str(number), center=True)
x = c.get_image_grayscale()

# little things:
# convert to float32, and convert 
print(x.shape, x.dtype)
x = torch.tensor(x, dtype=torch.float32).view(INPUT_SHAPE).to(device)
print(x.shape, x.dtype)

predicted = predict(model, x)
c.show(title=f'Predicted number: {predicted}', size=(512, 512))

Disruption, first idea: how about we invert the colours? We do that by adding: `1.0 - c.get_image_grayscale()` (our pixel values lie between 0 and 1.

In [ ]:
number = np.random.randint(0, 10)
c = canvas.Canvas(28, 28)
c.background(255)
c.fill(0)
c.text_size(26)
c.text([c.width/2, c.height/2 + 9], str(number), center=True)

# test: rotation?
# c.translate(c.width/2, c.height/2 + 7)
# c.rotate(torch.rand(1).item() * 2 * math.pi) # random rotation from 0 to 2 pi
# c.text([0, 0], str(number), center=True)

x = 1.0 - c.get_image_grayscale() # Inverted (note: this array has already values in [0,1], no need to divide by 255)

x = torch.tensor(x, dtype=torch.float32).view(INPUT_SHAPE).to(device)

predicted = predict(model, x)
c.show(title=f'Predicted number: {predicted}', size=(512, 512))

### Ideas for exploration

- Creatively disrupt the image, keeping it recognizable to a human, but causing the model to produce an incorrect prediction. You could add random dots, or patches, for instance. Or simply create an array of random numbers of the same size as the image and add it to the image.
- Try to do this in steps, e.g. incrementally adding modifications to the image and observing when and how it stops being recongized by the model.
- Briefly discuss the steps you are taking, taking advantage of the hybrid markdown/code format of the notebook.

Make sure to display the images you are creaating!

You may want to work with the `Canvas` object directly, using some tools demonstrated in the relevant notebook, in which case you should keep in mind that you are only producing grayscale images and that the images have size 28x28.

Otherwise you might as well work by preparing images externally (e.g. by hand, or using p5js) and then loading these as we have seen earlier for the image of a four. If you take this approach, make sure you start from an image that is consistently recognizable to a human as a given number and correctly classified by the model as that same number.

## 2. Generate

Here is a simple example that looks like a `0`, and usually gets classified as one.

In [ ]:
c = canvas.Canvas(28, 28)
c.background(0)

c.no_stroke()
for t in np.linspace(1, 0.2, 5):
    c.fill(255*t)
    c.circle([c.width/2, c.height/2], 10*t)

x = c.get_image_grayscale()

x = torch.tensor(x, dtype=torch.float32).view(INPUT_SHAPE).to(device)

predicted = predict(model, x)
c.show(title=f'Predicted number: {predicted}', size=(512, 512))

This most interesting when not using the text function any more, but rather using the drawing abilities of canvas.

Try different numbers!

**Also**, try shapes that *really do not look like numbers* to us, and see what happens.

As before, a ConvNet will probably perform better than a plain Dense net.

### Note

If you trained a net on FashionMNIST, you can do the same thing but with pieces of clothing! (The images must always be b&w, 28*28!).